In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from tqdm import tqdm

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the CPU


In [6]:
#training_data_glove = np.load("training_data_glove.npy", allow_pickle=True)
training_data_fasttext = np.load("training_data_fasttext.npy", allow_pickle=True)
DIMENSION = 300

In [28]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(DIMENSION, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 256)
        self.fc5 = nn.Linear(256, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.softmax(x, dim=1)

In [29]:
tr_size = round(len(training_data_fasttext)*0.8)
tr_set = training_data_fasttext[:tr_size]
te_set = training_data_fasttext[tr_size:]

In [30]:
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=300, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=2, bias=True)
)


In [31]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(0, len(te_set), BATCH_SIZE)):
            batch_X = [x for x, y_ in te_set[i:i+BATCH_SIZE]]
            batch_y = [y_ for x, y_ in te_set[i:i+BATCH_SIZE]]

            tensor_batch_X = torch.tensor(batch_X).float().view(-1, DIMENSION).to(device)
            tensor_batch_y = torch.tensor(batch_y).float().to(device)
            batch_out = net(tensor_batch_X)

            out_maxes = [torch.argmax(val) for val in batch_out]
            target_maxes = [torch.argmax(val) for val in tensor_batch_y]
            
            for i, j in zip(out_maxes, target_maxes):
                if i == j:
                    correct +=1
                total += 1
    return round(correct/total, 3)

In [34]:
BATCH_SIZE = 1
EPOCHS = 10
def train(net):
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss = 1000000000
    test_accuracy_at_each_epoch = []
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(tr_set), BATCH_SIZE)):
            batch_X = np.array([x for x, y_ in tr_set[i:i+BATCH_SIZE]])
            batch_y = np.array([y_ for x, y_ in tr_set[i:i+BATCH_SIZE]])

            tensor_batch_X = torch.tensor(batch_X).float()
            tensor_batch_y = torch.tensor(batch_y).float()

            tensor_batch_X, tensor_batch_y = tensor_batch_X.to(device), tensor_batch_y.to(device)

            net.zero_grad()

            output = net(tensor_batch_X.view(-1, DIMENSION))
            loss = loss_function(output, tensor_batch_y)
            loss.backward()
            optimizer.step()
      
        accuracy = test(net)
        test_accuracy_at_each_epoch.append(accuracy)
        print(f"\nLoss: {loss}")
        print(f"\nAccuracy: {accuracy}")
    return test_accuracy_at_each_epoch

In [35]:
test_accuracy_epoch = train(net)

100%|█████████████████████████████████| 454096/454096 [02:06<00:00, 3592.05it/s]



Loss: 1.0043319630538127e-14

Accuracy: 0.753


100%|█████████████████████████████████| 454096/454096 [02:01<00:00, 3740.52it/s]



Loss: 6.127363860741752e-08

Accuracy: 0.773


100%|█████████████████████████████████| 454096/454096 [02:09<00:00, 3513.97it/s]



Loss: 0.0

Accuracy: 0.773


100%|█████████████████████████████████| 454096/454096 [02:08<00:00, 3539.81it/s]



Loss: 3.3782837447496585e-37

Accuracy: 0.776


 46%|█████████████▎               | 835877/1816386 [1:25:16<1:40:02, 163.36it/s]


KeyboardInterrupt: 

In [ ]:
print(test_accuracy_epoch)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(list(range(1,EPOCHS+1)), test_accuracy_epoch)
plt.show()

In [ ]:
torch.save(net.state_dict(), "nn_dict")
torch.save(net, "nn_object")